In [1]:
import numpy as np
import pandas as pd
import os
import pickle

train_path = 'train/'
dir_list = os.listdir(path=train_path)

In [2]:
datafiles = []
labelfiles = []

for file in dir_list:
    if file[-8:] == 'data.csv':
        datafiles.append(train_path+file)
    else:
        labelfiles.append(train_path+file)
        #df = pd.read_csv(train_path+file, index_col=0)
        #print(df.head())
        #print(df.shape)

#5/0
file_starts = [i[:-8] for i in datafiles]



In [4]:

file_lengths = {}
dfs = {}
for i, file in enumerate(file_starts):
    if i%10==0:
        print(str(i), "files read")
    df = pd.read_csv(file+'data.csv', index_col=0)
    #print(df.head())
    file_lengths[file] = df.shape[0]
    #print(df.shape)
    dfs[file] = df




0 files read
10 files read
20 files read
30 files read
40 files read
50 files read
60 files read
70 files read
80 files read
90 files read


In [5]:
labeldfs={}
for i, file in enumerate(file_starts):
    if i%10==0:
        print(str(i), "files read")
    df = pd.read_csv(file+'events.csv', index_col=0)
    #print(df.head())
    #print(df.shape)
    labeldfs[file] = df

0 files read
10 files read
20 files read
30 files read
40 files read
50 files read
60 files read
70 files read
80 files read
90 files read


In [6]:
labeldf = pd.concat(list(labeldfs.values()))

In [7]:
fulldf = pd.concat(list(dfs.values()))

Generate all possible combinations of hand motions

In [8]:
def to_binary(x):
    rep = np.zeros(6)
    power = 5
    while x>0:
        if x>=2**power:
            x -= 2**power
            rep[power] = 1
        power -= 1
    return rep

events = np.zeros((2**6, 6))
for i in range(2**6):
    events[i] = to_binary(i)

Generate covariance matrices for all possible 'events', aka hand motion combinations

In [127]:
event_covmats = np.empty((64,32,32), dtype=np.float32)
event_means = np.empty((64,32), dtype=np.float32)
for i, ev in enumerate(events):
    if i%5==0 or i==1:
        print(i, "samples computed")
    wh = labeldf.where(labeldf == ev)
    inds = wh.index
    #print(inds)
    sampledf = fulldf.loc[inds]
    #print(sampledf)
    samples = np.array(sampledf).astype(np.int32)
    if len(samples)>1000000:
        rand_inds = np.random.choice(len(samples), size=1000000)
        samples = samples[rand_inds]
    
    del sampledf
    #print(samples)
    if len(samples>1):
        covmat = np.cov(samples.T)
        event_covmats[i,:,:] = covmat
        event_means[i,:] = np.mean(samples.T, axis=1)
    else:
        print('No samples for event', ev)
        event_covmats[i,:,:] = np.eye(32)
        event_means[i,:] = np.zeros(32)
    
    

0 samples computed
1 samples computed
5 samples computed
10 samples computed
15 samples computed
20 samples computed
25 samples computed
30 samples computed
35 samples computed
40 samples computed
45 samples computed
50 samples computed
55 samples computed
60 samples computed


In [132]:
np.save('eeg_event_covmats.npy', event_covmats)

In [133]:
np.save('eeg_event_means.npy', event_means)

generate covariance matrices for 'sub_events', which are individual motions being zero or one

In [134]:
sub_event_covmats = np.empty((6,2,32,32), dtype=np.float32)
sub_event_means = np.empty((6,2,32), dtype=np.float32)


for i in range(6):
    
    print(i, "samples computed")
    wh = labeldf.where(labeldf[labeldf.columns.values] == 1)
    inds = wh.index
    #print(inds)
    sampledf = fulldf.loc[inds]
    #print(sampledf)
    samples = np.array(sampledf).astype(np.int32)
    if len(samples)>1000000:
        rand_inds = np.random.choice(len(samples), size=1000000)
        samples = samples[rand_inds]
    
    del sampledf
    #print(samples)
    if len(samples>1):
        covmat = np.cov(samples.T)
        sub_event_covmats[i,1,:,:] = covmat
        sub_event_means[i,1,:] = np.mean(samples, axis=0)
    else:
        print('No samples for event', ev)
        sub_event_covmats[i,1,:,:] = np.eye(32)

for i in range(6):
    
    print(i+6, "samples computed")
    wh = labeldf.where(labeldf[labeldf.columns.values] == 0)
    inds = wh.index
    #print(inds)
    sampledf = fulldf.loc[inds]
    #print(sampledf)
    samples = np.array(sampledf).astype(np.int32)
    if len(samples)>1000000:
        rand_inds = np.random.choice(len(samples), size=1000000)
        samples = samples[rand_inds]
    
    del sampledf
    #print(samples)
    if len(samples>1):
        covmat = np.cov(samples.T)
        sub_event_covmats[i,0,:,:] = covmat
        sub_event_means[i,0,:] = np.mean(samples, axis=0)

    else:
        print('No samples for event', ev)
        sub_event_covmats[i,0,:,:] = np.eye(32)
    

0 samples computed
1 samples computed
2 samples computed
3 samples computed
4 samples computed
5 samples computed
6 samples computed
7 samples computed
8 samples computed
9 samples computed
10 samples computed
11 samples computed


In [135]:
np.save('eeg_sub_event_covmats.npy', sub_event_covmats)

In [136]:
np.save('eeg_sub_event_means.npy', sub_event_means)

Compute and save inverses for use in gmm

In [137]:
event_covmat_invs = [np.linalg.inv(mat) for mat in event_covmats]

In [138]:
sub_event_covmat_invs = [[np.linalg.inv(mat), np.linalg.inv(mmat)] for [mat,mmat] in sub_event_covmats]

In [139]:
np.save('eeg_sub_event_covmat_invs.npy', sub_event_covmat_invs)

In [140]:
np.save('eeg_event_covmat_invs.npy', event_covmat_invs)

In [141]:
np.save('eeg_events.npy', events)

test using maha distance for assigning samples to classify samples (works very poorly

In [148]:
def test_cov_clusters(n, print_freq=100):
    inds = np.random.choice(len(fulldf),size=n)
    names = fulldf.index[inds]
    samples = fulldf.loc[names]
    names = samples.index
    targets = labeldf.loc[names]
    targs2 = np.empty((n,6), dtype=np.uint8)
    if len(targets)==len(samples):
        print('sample and target lengths match')
    
    preds = np.empty((n,6), dtype=np.uint8)
    for i, name in enumerate(names):
        if i%print_freq==0:
            print(i, 'preds made')
        vec = np.array(samples.loc[name])
        
        targ = np.array(targets.loc[name])
        if vec.shape != (32,):
            vec=vec[0]
            targ=targ[0]
        shortest, best = np.inf, None
        for j in range(len(event_covmat_invs)):
            mat = event_covmat_invs[j]
            mu = event_means[j]
            diff = vec-mu
            dist = (diff.T@mat@diff)**0.5
            if dist<shortest:
                shortest=dist
                best=j
        preds[i] = events[j]
        targs2[i] = targ
    correct = float(0)
    error = float(0)
    for i,pred in enumerate(preds):
        t = targs2[i]
        if np.array_equal(pred,t):
            correct += 1
        else:
            error += np.linalg.norm(pred-t)
    return preds, correct, error
        

In [149]:
test_cov_clusters(1000)

sample and target lengths match
0 preds made
100 preds made
200 preds made
300 preds made
400 preds made
500 preds made
600 preds made
700 preds made
800 preds made
900 preds made


(array([[1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1],
        ...,
        [1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1]], dtype=uint8), 0.0, 2406.9804321263928)

In [144]:
def test_cov_clusters2(n, print_freq=100):
    inds = np.random.choice(len(fulldf),size=n)
    names = fulldf.index[inds]
    samples = fulldf.loc[names]
    names = samples.index
    targets = labeldf.loc[names]
    targs2 = np.empty((n,6), dtype=np.uint8)
    if len(targets)==len(samples):
        print('yayyy')
    
    preds = np.empty((n,6), dtype=np.uint8)
    for i, name in enumerate(names):
        if i%print_freq==0:
            print(i, 'preds made')
        vec = np.array(samples.loc[name])
        targ = np.array(targets.loc[name])
        for j in range(len(sub_event_covmat_invs)):
            (mat0,mat1) = sub_event_covmat_invs[j]
            (mu0,mu1) = sub_event_means[j]
            diff0=vec-mu0
            diff1=vec-mu1
            dist0 = (diff0.T@mat0@diff0)**0.5
            dist1 = (diff1.T@mat1@diff1)**0.5
            if dist0<dist1:
                preds[i,j]=0
            else:
                preds[i,j]=1
            targs2[i] = targ
    correct = float(0)
    error = float(0)
    norm = float(0)
    prednorm = float(0)
    for i,pred in enumerate(preds):
        t = targs2[i]
        if np.array_equal(pred,t):
            correct += 1
        else:
            error += np.linalg.norm(pred-t)
        norm += np.linalg.norm(t)
        prednorm += np.linalg.norm(pred)
    return preds, correct, error/n, norm/n, prednorm/n
        

In [145]:
test_cov_clusters2(1000)

yayyy
0 preds made
100 preds made
200 preds made
300 preds made
400 preds made
500 preds made
600 preds made
700 preds made
800 preds made
900 preds made


(array([[1, 0, 1, 0, 1, 0],
        [1, 0, 1, 0, 1, 0],
        [1, 0, 0, 0, 1, 0],
        ...,
        [0, 1, 1, 1, 1, 1],
        [1, 0, 0, 1, 1, 0],
        [1, 1, 0, 0, 1, 0]], dtype=uint8),
 1.0,
 17.59234387244542,
 0.1623799186067802,
 1.9622870743911935)

In [150]:
labeldf.head()

,HandStart,FirstDigitTouch,BothStartLoadPhase,LiftOff,Replace,BothReleased
id,,,,,,
subj8_series4_0,0,0,0,0,0,0
subj8_series4_1,0,0,0,0,0,0
subj8_series4_2,0,0,0,0,0,0
subj8_series4_3,0,0,0,0,0,0
subj8_series4_4,0,0,0,0,0,0


In [4]:
X = np.load('eeg_event_evals.npy')
X[0]

array([ 12291.18 ,  12702.946,  14292.849,  15292.013,  15696.541,
        16811.916,  18231.11 ,  18425.246,  18826.678,  19966.84 ,
        21492.184,  23101.332,  23948.854,  26927.422,  28953.607,
        30318.188,  32186.611,  33003.324,  36582.613,  38945.11 ,
        46248.49 ,  48992.87 ,  50892.08 ,  60018.69 ,  65010.516,
        71122.375,  81071.45 , 101630.31 , 128328.75 , 174817.81 ,
       472340.66 , 889802.25 ], dtype=float32)